In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/Shareddrives/CS229Buddies/project')

In [3]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import collections
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve, f1_score, plot_precision_recall_curve
from scipy.stats import pearsonr
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import operator
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [4]:
GO=pd.read_csv("gseapy_results/GO_Biological_Process_2018.human.enrichr.reports.txt", sep="\t")

In [41]:
import collections

In [49]:
GO["Genes"].values[0]

'IFNA5;ITGB1;IFNA4;IFNA7;IFNA6;IFNA1;FLT3;IFNA2;IFNA8;DCAF1;LYL1;HHEX;RAG2;JAK3;RAG1;IL10;TPD52;IL11;CR2;IFNA14;ITGA4;IFNA16;NFAM1;IFNA17;ADAM17;MSH2;IFNE;KIT;IFNK;IFNA10;HDAC4;HDAC5;CEBPG;HDAC9;CD79B;ADGRG3;CD79A;PLCG2;DNAJB9;FNIP1;NHEJ1;IFNA21;VCAM1;IFNB1;IFNW1;IL4;KLF6;CLCF1;TCF3;PTPN2'

In [77]:
GO_map=pd.Series(GO["Genes"].values,index=GO["Term"]).to_dict()
for k, v in GO_map.items():
  GO_map[k]=set(v.split(";"))&set(control_df.columns)
GO_map

{'B cell differentiation (GO:0030183)': {'CLCF1',
  'HDAC9',
  'IL11',
  'ITGA4',
  'KIT',
  'MSH2',
  'TCF3',
  'VCAM1'},
 'regulation of cellular metabolic process (GO:0031323)': {'CYP1B1',
  'DKK1',
  'FASN',
  'FOXM1',
  'PDK4',
  'PID1',
  'STUB1'},
 'negative regulation of phosphatase activity (GO:0010923)': {'ELFN1',
  'ELL',
  'FKBP1A',
  'KNL1',
  'MASTL',
  'MYO1D',
  'PKMYT1',
  'PLEKHF1',
  'PPP1R12B',
  'PPP1R12C',
  'PPP1R26',
  'PPP1R35',
  'PPP1R37',
  'SH2D4A',
  'SPOCD1',
  'SPRED1',
  'SYTL2',
  'WDR81'},
 'cartilage development (GO:0051216)': {'BMP2',
  'BMP4',
  'CD44',
  'CHI3L1',
  'FGF2',
  'GDF5',
  'HMGA2',
  'HOXA11',
  'HOXD3',
  'HYAL2',
  'HYAL3',
  'ITGB8',
  'MEF2C',
  'NOV',
  'OSR1',
  'OSR2',
  'PKD1',
  'RUNX1',
  'RUNX3',
  'SOX9',
  'SULF1',
  'SULF2',
  'TGFB1',
  'WNT5B'},
 'regulation of potassium ion transport (GO:0043266)': {'ADRA1D',
  'ADRA2A',
  'ANK2',
  'DLG1',
  'FHL1',
  'KCNIP3'},
 'response to dsRNA (GO:0043331)': {'PMAIP1', 'RFTN1', 

In [52]:
from scipy import stats

In [81]:
control_df=control_df.loc[:,~control_df.columns.duplicated()]

In [73]:
gene_corr=dict()
for gene in control_df.columns:
  gene_corr[gene]=stats.pearsonr(control_df[gene],np.array(ages))[0]

In [112]:
for k, v in GO_map.items():
  corrs=[]
  for gene in v:
    corrs.append(gene_corr[gene])
  df=pd.DataFrame(data=[v,corrs]).T
  df[1]=df[1].astype("float")
  mean=np.array(corrs).mean()
  df=df.sort_values(by=1, ascending=(np.array(corrs).mean()<0))
  print(df)
  a=[]
  a.append(control_df.loc[:,df.iloc[0,0]])
  print(a)

  
  break

       0         1
5   TCF3 -0.465552
6   IL11 -0.322188
7  CLCF1 -0.224876
1   MSH2 -0.162964
3  VCAM1 -0.069183
2  HDAC9  0.018147
0    KIT  0.267175
4  ITGA4  0.455227
[101_19yr_Female_Caucasian    12.415
102_19yr_Male_Caucasian      10.714
103_19yr_Male_Caucasian      13.013
104_19yr_Male_Caucasian        12.1
105_20yr_Male_Caucasian      12.556
                              ...  
200_8yr6mos_M_Unknown        13.166
201_6ys11mos_F_Unknown       14.516
202_5yr0mos_F_Unknown        11.094
203_8yr10mos_M_Unknown         8.51
204_3yr0mos_F_Unknown        10.235
Name: TCF3, Length: 143, dtype: object]


In [107]:
df

,1
0,
TCF3,-0.465552
IL11,-0.322188
CLCF1,-0.224876
MSH2,-0.162964
VCAM1,-0.069183
HDAC9,0.018147
KIT,0.267175
ITGA4,0.455227


In [80]:
df = pd.read_csv('data/GSE113957_fpkm.txt', sep = '\t')
df = df.T
transcript_ids = list(df.loc['Transcript ID'])

df = df.drop(['Transcript ID', 'chr', 'start', 'end', 'strand', 'Length', 'Copies', 'Annotation/Divergence'], axis=0)
df['pids'] = [column.lower().split('_')[0] for column in df.index]
hgps = set(['139', '141', '197', '198', '199', '200', '201', '202', '203', '204'])
control_df = df[~df.pids.isin(hgps)]
hgps_df = df[df.pids.isin(hgps)]
control_df = control_df.drop(['pids'], axis = 1)
hgps_df = hgps_df.drop(['pids'], axis = 1)

control_df.columns = transcript_ids
hgps_df.columns = transcript_ids

total_df = control_df.append(hgps_df)
annot=pd.read_csv("data/uniprot.tab", sep='\t')
annot=pd.read_csv("data/uniprot.tab", sep='\t')
name_d=pd.Series(annot["Gene names"].values,index=annot["yourlist:M202105245C475328CEF75220C360D524E9D456CE20BD46G"]).to_dict()
n=dict()
for k,v in name_d.items():
  if str(v)!="nan" and str(k)!="nan":
    n[k]=v.split(" ")[0]
control_df=total_df[[x for x in control_df.columns if x  in n.keys()]]
control_df.columns=[n[x] for x in control_df.columns if x  in n.keys()]

In [35]:
def filter_genes(X):
  genes=[]
  for col in X.columns:
    vec=np.array(X[col])
    if vec.max()> 5 and vec.max()-vec.min()>5*vec.min():
      genes.append(col)
  return X[genes]

In [36]:
control_df=filter_genes(control_df)

In [113]:
for col in control_df.columns:
  # print(col)
  # print(np.array(total_df[col]).std())
  control_df[col]=(control_df[col]-np.array(control_df[col]).mean())/np.array(control_df[col]).std()


ZeroDivisionError: ignored

In [40]:
labels = control_df.index
ages = [int(label.lower().split('_')[1].split('y')[0]) for label in labels]
sex = [int(label.lower().split('_')[2] == 'male') for label in labels]
race = [label.lower().split('_')[3] for label in labels]

In [29]:
for index, row in GO.iterrows

SyntaxError: ignored

In [9]:
from scipy.stats import zscore

In [21]:
total_df["C12orf74"]

101_19yr_Female_Caucasian    0
102_19yr_Male_Caucasian      0
103_19yr_Male_Caucasian      0
104_19yr_Male_Caucasian      0
105_20yr_Male_Caucasian      0
                            ..
200_8yr6mos_M_Unknown        0
201_6ys11mos_F_Unknown       0
202_5yr0mos_F_Unknown        0
203_8yr10mos_M_Unknown       0
204_3yr0mos_F_Unknown        0
Name: C12orf74, Length: 143, dtype: object